In [10]:
import os

In [11]:
%pwd

'c:\\Users\\haari\\OneDrive\\Desktop\\aiml 2026\\Kidney-Disease-Classification-Deep-Learning-Project\\research\\research\\templates'

In [12]:
os.chdir("../../..")

In [13]:
%pwd

'c:\\Users\\haari\\OneDrive\\Desktop\\aiml 2026\\Kidney-Disease-Classification-Deep-Learning-Project'

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareBaseModelConfig:   
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_classes: int
    params_weights: str
    params_include_top: bool



In [15]:
import sys
from pathlib import Path

# get project root (folder that contains 'src')
PROJECT_ROOT = Path(r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project")

# add src to python path
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("PYTHONPATH set correctly ✅")
print(sys.path[0])


PYTHONPATH set correctly ✅
C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\src


In [16]:
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *


In [17]:
import sys
from pathlib import Path

# get project root (folder that contains 'src')
PROJECT_ROOT = Path(r"C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project")

# add src to python path
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

print("PYTHONPATH set correctly ✅")
print(sys.path[0])


PYTHONPATH set correctly ✅
C:\Users\haari\OneDrive\Desktop\aiml 2026\Kidney-Disease-Classification-Deep-Learning-Project\src


In [18]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=params.image_size,
            params_learning_rate=params.learning_rate,
            params_classes=params.classes,
            params_weights=params.weights,
            params_include_top=params.include_top
        )

        return prepare_base_model_config


In [19]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [20]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = None

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(
            path=self.config.base_model_path,
            model=self.model
        )

    @staticmethod
    def _prepare_full_model(
        model,
        classes,
        freeze_all,
        freeze_till,
        learning_rate
    ):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
         
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=learning_rate
            ),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(
            path=self.config.updated_base_model_path,
            model=full_model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [21]:
config = ConfigurationManager()
prepare_base_model_config = config.get_prepare_base_model_config()

prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
prepare_base_model.get_base_model()
prepare_base_model.update_base_model()


2026-02-06 21:05:38,403 : INFO : cnnClassifier : yaml file: configs\config.yaml loaded successfully
2026-02-06 21:05:38,405 : INFO : cnnClassifier : yaml file: params.yaml loaded successfully
2026-02-06 21:05:38,407 : INFO : cnnClassifier : created directory at: artifacts
2026-02-06 21:05:38,856 : WARNING : tensorflow : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                  

c:\Users\haari\.conda\envs\kidney\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
